Importing libraries

In [2]:
import pandas as pd
from google.cloud import vision, storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\storage_key.json"

Reading csv file containing product_name and url

In [46]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\Product_DB.csv")
data.shape

(60, 2)

Storing url column in data_url

In [47]:
data_url = data['product_url']
data_url

0     https://m.media-amazon.com/images/I/51MiYR1te6...
1     https://m.media-amazon.com/images/I/71cy1dbMh5...
2     https://m.media-amazon.com/images/I/71xWtLhH2G...
3     https://m.media-amazon.com/images/I/61C+Y9zoAQ...
4     https://m.media-amazon.com/images/I/71YG7EClYf...
5     https://m.media-amazon.com/images/I/81ATe15IyH...
6     https://m.media-amazon.com/images/I/61OBcY37KX...
7     https://m.media-amazon.com/images/I/710+f7XX2F...
8     https://m.media-amazon.com/images/I/71u2XOFXAI...
9     https://m.media-amazon.com/images/I/61U0BBQYhr...
10    https://m.media-amazon.com/images/I/71IQc8cAIQ...
11    https://m.media-amazon.com/images/I/71fKZrHSO9...
12    https://m.media-amazon.com/images/I/71FAiQfnqS...
13    https://m.media-amazon.com/images/I/61z4pzDQRO...
14    https://m.media-amazon.com/images/I/71N-eWQzzj...
15    https://m.media-amazon.com/images/I/51-FneXbwr...
16    https://m.media-amazon.com/images/I/51oqHcH9mc...
17    https://m.media-amazon.com/images/I/61a9Dd

Function to check if url contains a image or not

In [48]:
import requests
def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   r = requests.head(image_url)
   if r.headers["content-type"] in image_formats:
      # print(r.headers["content-type"])
      return True
   return False

In [5]:
import webcolors
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

requested_colour = (119, 172, 152)
actual_name, closest_name = get_colour_name(requested_colour)

In [7]:

def detect_properties_uri(uri):
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    # print("Properties:",props.dominant_colors.colors)
    # scr = [col for col in props.dominant_colors.colors]
    # print(scr)
    for color in props.dominant_colors.colors:
        try:
            r = int(color.color.red)
            g = int(color.color.green)
            b = int(color.color.blue)
            print(f"color: {get_colour_name((r,g,b))}")
        except Exception as e:
            print("gone from here",e)
            continue
        
detect_properties_uri(data_url[1])

color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'rosybrown')
color: (None, 'whitesmoke')
color: (None, 'darkslategray')
color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'gray')
color: (None, 'black')
color: (None, 'darkslategray')


Empty String variable to store results

In [49]:
txt = ""

Function to extract information from image

In [52]:
def extract_info(uri):
    global txt
    try:
        if(is_url_image(uri)==True):
            client = vision.ImageAnnotatorClient()
            img = vision.Image()
            img.source.image_uri = uri
            
            #For retriving Brand
            # res_logo = client.logo_detection(img)
            # logos = res_logo.logo_annotations
            # logo = [log.description for log in logos]
            # txt = txt + f"Brand: None, " if len(logo)==0 else txt + f"Brand: {logo[0]}, "
            
            #For retriving Labels
            res_label = client.label_detection(img)
            labels = res_label.label_annotations
            label = [lab.description for lab in labels]
            # txt = txt + f"Description: None, " if len(label)==0 else txt + f"Description: {str(label)}, "
            
            # #For retriving Text
            # res_text = client.document_text_detection(img)
            # texts = res_text.text_annotations
            # text = [t.description for t in texts]
            # txt = txt + f"Text: None, " if len(text)==0 else txt + f"Text: {text[0]}, "
            
            objects = client.object_localization(image=img).localized_object_annotations
            obj = [ob.name for ob in objects]
            
            txt = txt + f"{", ".join(label)}" if len(obj)==0 else txt + f"{", ".join(obj)}"
            
        
        else:
            txt = "Please enter a valid URL that contains Image"
    except Exception as e:
        txt = f"Please Enter a Valid URL of Image"
        return

Creating new dataframe to save results

In [53]:
result = data
result.head()

,product_name,product_url
0,Puma Unisex-Adult Smashic Casual Shoe,https://m.media-amazon.com/images/I/51MiYR1te6...
1,NIKE Womens W Revolution 7 Running Shoe,https://m.media-amazon.com/images/I/71cy1dbMh5...
2,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...
3,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...
4,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...


Iterating through function to extract features of image and adding it to result dataframe

In [54]:
for i in range(len(data_url)):
    txt = ""
    extract_info(data_url[i])
    result.loc[i, "objects_extracted"] = txt
    

Resultant Dataframe

In [55]:
result

,product_name,product_url,objects_extracted
0,Puma Unisex-Adult Smashic Casual Shoe,https://m.media-amazon.com/images/I/51MiYR1te6...,Shoe
1,NIKE Womens W Revolution 7 Running Shoe,https://m.media-amazon.com/images/I/71cy1dbMh5...,Shoe
2,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,Shoe
3,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,Shoe
4,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,Shoe
5,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,Shoe
6,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,Shoe
7,Bata Mens MaxUniform Dress Shoe,https://m.media-amazon.com/images/I/710+f7XX2F...,Shoe
8,SPARX Mens Sx0675g Walking Shoe,https://m.media-amazon.com/images/I/71u2XOFXAI...,Shoe
9,Campus Stack Sneakers for Men,https://m.media-amazon.com/images/I/61U0BBQYhr...,Shoe


Saving Dataframe to CSV file

In [4]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)

NameError: name 'result' is not defined

In [6]:
nnn = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
nnn.shape

(0, 3)

In [7]:
nnn.to_csv("gs://bucket-shreyash/Product_Data/Product_D.csv",index=False)

In [9]:
df= pd.read_csv("gs://bucket-shreyash/Product_Data/Product_D.csv")
df

,product_name,product_url,objects_extracted
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,['Shoe']
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,['Shoe']
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,['Shoe']
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,['Shoe']
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,['Shoe']
...,...,...,...
152,GORMERY 1/12 Scale Model Compatible with Kawasaki,https://m.media-amazon.com/images/I/41zDXFipUm...,"['Motorcycle', 'Tire', 'Tire', 'Wheel', 'Wheel']"
153,Nexus Product Bike for Kids Toy R3 Bike,https://m.media-amazon.com/images/I/71HxhWJlza...,"['Tire', 'Wheel', 'Motorcycle', 'Wheel', 'Tire..."
154,PLUSPOINT Diecast Motorcycle Toy Ducati Bike,https://m.media-amazon.com/images/I/71NNd7bV8P...,"['Motorcycle', 'Wheel', 'Tire', 'Wheel', 'Tire..."
155,Bajaj Frore Neo Table Fan 400 MM,https://m.media-amazon.com/images/I/61uCb3XScb...,['Mechanical fan']


In [12]:
query='shoe'

In [16]:
if df['product_name'].str.lower().str.contains(query).any():
    name = df[df['product_name'].str.lower().str.contains(query)]['product_name'].tolist()
    print(name)
    link = df[df['product_name'].str.lower().str.contains(query)]['product_url'].tolist()

['Puma Womens Kendall WNS Running Shoe', "Nivia Men's Combat 2.0 Basketball Shoes", 'adidas Mens Amalgo M Running Shoe', 'Red Tape Casual Sneaker Shoes for Men', 'Bata Mens MaxUniform Dress Shoe', 'SPARX Mens Sx0675g Walking Shoe']


In [9]:

namee = a[a['product_name'].map(lambda x: x.lower()).str.contains('shoes').any()]['product_name'].tolist()
link = a[a['product_name'].map(lambda x: x.lower()).str.contains('shoes')]['product_url'].tolist()

In [10]:
namee

['Leader Spyder 27.5T MTB Cycle/Bike',
 'Leader Scout MTB 26T Mountain Bicycle/Bike',
 'Amitasha Pull Back Motorbike High-Speed Racing Bike',
 'Baybee R7 Battery Operated Bike for Kids',
 'Nexus Product Bike for Kids Toy R3 Bike',
 'PLUSPOINT Diecast Motorcycle Toy Ducati Bike']

In [25]:
d = a['product_name'].map(lambda x: x.lower())

In [26]:
d

0                           sparx men's sc0162g sneakers
1                   puma womens kendall wns running shoe
2                nivia men's combat 2.0 basketball shoes
3                      adidas mens amalgo m running shoe
4                  red tape casual sneaker shoes for men
                             ...                        
152    gormery 1/12 scale model compatible with kawasaki
153              nexus product bike for kids toy r3 bike
154         pluspoint diecast motorcycle toy ducati bike
155                 solakaka sm600 wireless gaming mouse
156    zebronics zeb-blaze rgb gaming mouse pad with ...
Name: product_name, Length: 157, dtype: object

In [78]:
ob = nnn['objects_extracted']

In [91]:
import ast

e = {'Television'}
for i in ob:
    i = set(ast.literal_eval(i))
    if len(e.intersection(i)) !=0:
        print(i)


{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}


Uploading csv file to Cloud Stroage

In [2]:
storage_cli = storage.Client()
bucket = storage_cli.get_bucket("bucket-shreyash")
blob = bucket.blob("Extracted_features_from_image.csv")
blob.upload_from_filename("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
print("completed")

completed
